Named Entity Recognization

In [ ]:
import spacy
from nltk import sent_tokenize

In [ ]:
!python -m spacy download en_core_web_trf

Load Dataset

In [ ]:
import os 
import sys
import pathlib
folder_path = pathlib.Path().parent.resolve()
sys.path.append(os.path.join(folder_path, '../'))
from utils import load_subtitles_dataset

In [ ]:
dataset_path = "../data/Subtitles/"
df = load_subtitles_dataset(dataset_path)

In [ ]:
df.head()

In [ ]:
sample_script = df.iloc[0]['script']
sample_script

In [ ]:
sentences = sent_tokenize(sample_script)

In [ ]:
sentences = sentences[60:90]

In [ ]:
sentence = ".".join(sentences)

In [ ]:
sentence

#Run Model

In [ ]:
doc = nlp_model(sentence)

In [ ]:
doc.ents

In [ ]:
for entity in doc.ents:
    print(entity, entity.label_)

In [ ]:
def get_ners_inference(script):
    script_sentences = sent_tokenize(script)

    ner_output = []

    for sentence in script_sentences:
        doc = nlp_model(sentence)
        ners = set()
        for entity in doc.ents:
            if entity.label_ =="PERSON":
                full_name = entity.text
                first_name = entity.text.split(" ")[0]
                first_name = first_name.strip()
                ners.add(first_name)
        ner_output.append(ners)

    return ner_output

In [ ]:
df = df.head(10)

In [ ]:
df

In [ ]:
df['ners'] = df['script'].apply(get_ners_inference)

In [ ]:
df

#Character Network

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network

In [ ]:
def generate_character_network(df):

    windows=10
    entity_relationship = []

    for row in df['ners']:
        previous_entities_in_window = []

        for sentence in row:
            previous_entities_in_window.append(list(sentence))
            previous_entities_in_window = previous_entities_in_window[-windows:]

            # Flatten 2D List into 1D List
            previous_entities_flattened = sum(previous_entities_in_window, [])

            for entity in sentence:
                for entity_in_window in previous_entities_flattened:
                    if entity != entity_in_window:
                        entity_relationship.append(sorted([entity, entity_in_window]))
    
    relationship_df = pd.DataFrame({'value': entity_relationship})
    relationship_df['source'] = relationship_df['value'].apply(lambda x: x[0])
    relationship_df['target'] = relationship_df['value'].apply(lambda x: x[1])
    relationship_df = relationship_df.groupby(['source', 'target']).count().reset_index()
    relationship_df = relationship_df.sort_values('value', ascending=False)

    return relationship_df

In [ ]:
relationship_df = generate_character_network(df)

In [ ]:
relationship_df

In [ ]:
relationship_df = relationship_df.sort_values('value', ascending=False)
relationship_df = relationship_df.head(200)

In [ ]:
G = nx.from_pandas_edgelist(
    relationship_df, 
    source='source', 
    target='target', 
    edge_attr='value',
    create_using=nx.Graph()
)

net = Network(notebook=True, width="1000px", height="700px", bgcolor="#222222", font_color="white", cdn_resources="remote")
node_degree = dict(G.degree)

nx.set_node_attributes(G, node_degree, 'size')
net.from_nx(G)
net.show("naruto.html")